# ENDF Data Retrieval and Caching in PLEIADES

In [1]:
import os
from pathlib import Path
from pleiades.nuclear.manager import NuclearDataManager, DataSource, EndfLibrary
from pleiades.utils.config import get_config
import logging

# Configure logging to see debug information
logging.basicConfig(level=logging.INFO)

In [2]:
config = get_config()
print(f"Cache directory: {config.nuclear_data_cache_dir}")
print(f"Available data sources: {config.nuclear_data_sources}")

# Create output directory for parameter files
output_dir = Path("./endf_test")
output_dir.mkdir(exist_ok=True)
print(f"Output directory: {output_dir.absolute()}")

Cache directory: /Users/8cz/.pleiades/nuclear_data
Available data sources: {'IAEA': 'https://www-nds.iaea.org/public/download-endf', 'NNDC': 'https://www.nndc.bnl.gov/endf/b8.0/data'}
Output directory: /Users/8cz/github.com/PLEIADES/notebook/tutorial/endf_test


In [3]:
nuclear_manager = NuclearDataManager()
print(f"Default ENDF library: {nuclear_manager.default_library}")

# List cache directories
cache_root = config.nuclear_data_cache_dir
print("Cache structure:")
if cache_root.exists():
    for source in os.listdir(cache_root):
        source_dir = cache_root / source
        if source_dir.is_dir():
            print(f"  Source: {source}")
            for library in os.listdir(source_dir):
                library_dir = source_dir / library
                if library_dir.is_dir():
                    file_count = len(list(library_dir.glob("*.dat")))
                    print(f"    Library: {library} ({file_count} files)")

Default ENDF library: EndfLibrary.ENDF_B_VIII_0
Cache structure:
  Source: DataSource.NNDC
    Library: EndfLibrary.JENDL_5 (0 files)
    Library: EndfLibrary.TENDL_2021 (0 files)
    Library: EndfLibrary.JEFF_3_3 (0 files)
    Library: EndfLibrary.ENDF_B_VIII_1 (0 files)
    Library: EndfLibrary.ENDF_B_VIII_0 (0 files)
    Library: EndfLibrary.CENDL_3_2 (0 files)
  Source: DataSource.IAEA
    Library: EndfLibrary.JENDL_5 (0 files)
    Library: EndfLibrary.TENDL_2021 (0 files)
    Library: EndfLibrary.JEFF_3_3 (0 files)
    Library: EndfLibrary.ENDF_B_VIII_1 (0 files)
    Library: EndfLibrary.ENDF_B_VIII_0 (0 files)
    Library: EndfLibrary.CENDL_3_2 (0 files)


In [4]:
isotope_str = "U-238"  # Test with U-238
print(f"Testing with isotope: {isotope_str}")

# Get isotope info
isotope_info = nuclear_manager.isotope_manager.get_isotope_info(isotope_str)
print(f"Isotope details: {isotope_info}")

Testing with isotope: U-238
Searching for mass.mas20 in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info')}
Checking file: neutrons.list
Checking file: mass.mas20
Found file: /Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20
Searching for isotopes.info in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info')}
Checking file: neutrons.list
Checking file: mass.mas20
Checking file: isotopes.info
Found file: /Users/8cz/github.com/PLEI

In [5]:
%%time
# Download and extract resonance file (first time should download)
print("\nFirst download (should fetch from remote):")
output_path = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    source=DataSource.IAEA,
    use_cache=True
)
print(f"Output file: {output_path}")

INFO:pleiades.nuclear.manager:Downloading ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9237_92-U-238.zip



First download (should fetch from remote):


INFO:pleiades.nuclear.manager:Resonance parameters written to endf_test/092-U-238.B-VIII.0.par


Output file: endf_test/092-U-238.B-VIII.0.par
CPU times: user 106 ms, sys: 67.1 ms, total: 173 ms
Wall time: 2.06 s


In [6]:
cache_file_path = nuclear_manager._get_cache_file_path(
      source=DataSource.IAEA,
      library=EndfLibrary.ENDF_B_VIII_0,
      isotope=isotope_info,
      mat=isotope_info.material_number
  )
print(f"Cache file path: {cache_file_path}")
print(f"Exists in cache: {cache_file_path.exists()}")

if cache_file_path.exists():
    # Get file size
    file_size = cache_file_path.stat().st_size
    print(f"Cache file size: {file_size / 1024:.2f} KB")

    # Preview first few lines
    print("\nFirst 5 lines of cached file:")
    with open(cache_file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            print(line.strip())

Cache file path: /Users/8cz/.pleiades/nuclear_data/DataSource.IAEA/EndfLibrary.ENDF_B_VIII_0/n_9237_92-U-238.dat
Exists in cache: True
Cache file size: 15741.04 KB

First 5 lines of cached file:
Retrieved by E4-util: 2018/02/07,18:09:12                            1 0  0    0
9.223800+4 2.360058+2          1          1          0          19237 1451    1
0.000000+0 0.000000+0          0          0          0          69237 1451    2
1.000000+0 3.000000+7          0          0         10          89237 1451    3
0.000000+0 0.000000+0          0          0        566        2469237 1451    4


In [7]:
%%time

print("\nSecond download (should use cache):")
output_path2 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    source=DataSource.IAEA,
    use_cache=True
)
print(f"Output file: {output_path2}")

INFO:pleiades.nuclear.manager:Using cached ENDF data from /Users/8cz/.pleiades/nuclear_data/DataSource.IAEA/EndfLibrary.ENDF_B_VIII_0/n_9237_92-U-238.dat
INFO:pleiades.nuclear.manager:Resonance parameters written to endf_test/092-U-238.B-VIII.0.par



Second download (should use cache):
Output file: endf_test/092-U-238.B-VIII.0.par
CPU times: user 25.3 ms, sys: 8.78 ms, total: 34.1 ms
Wall time: 35.5 ms


In [8]:
%%time
print("\nTesting with ENDF-B-VIII.1 (different filename format):")

output_path3 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info,
    library=EndfLibrary.ENDF_B_VIII_1,
    output_dir=str(output_dir),
    source=DataSource.IAEA,
    use_cache=True
)
print(f"Output file: {output_path3}")

# Check the cache file path for VIII.1
cache_file_path2 = nuclear_manager._get_cache_file_path(
    source=DataSource.IAEA,
    library=EndfLibrary.ENDF_B_VIII_1,
    isotope=isotope_info,
    mat=isotope_info.material_number
)
print(f"VIII.1 cache file path: {cache_file_path2}")
print(f"VIII.1 exists in cache: {cache_file_path2.exists()}")

INFO:pleiades.nuclear.manager:Downloading ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.1/n/n_092-U-238_9237.zip



Testing with ENDF-B-VIII.1 (different filename format):


INFO:pleiades.nuclear.manager:Resonance parameters written to endf_test/092-U-238.B-VIII.1.par


Output file: endf_test/092-U-238.B-VIII.1.par
VIII.1 cache file path: /Users/8cz/.pleiades/nuclear_data/DataSource.IAEA/EndfLibrary.ENDF_B_VIII_1/n_092-U-238_9237.dat
VIII.1 exists in cache: True
CPU times: user 93.1 ms, sys: 55.4 ms, total: 148 ms
Wall time: 1.51 s


In [9]:
%%time

isotope_str2 = "U-235"
print(f"\nTesting with another isotope: {isotope_str2}")

isotope_info2 = nuclear_manager.isotope_manager.get_isotope_info(isotope_str2)
print(f"Isotope details: {isotope_info2}")

# Download and extract resonance file
output_path4 = nuclear_manager.download_endf_resonance_file(
    isotope=isotope_info2,
    library=EndfLibrary.ENDF_B_VIII_0,
    output_dir=str(output_dir),
    source=DataSource.IAEA,
    use_cache=True
)
print(f"Output file: {output_path4}")

INFO:pleiades.nuclear.manager:Downloading ENDF data from https://www-nds.iaea.org/public/download-endf/ENDF-B-VIII.0/n/n_9228_92-U-235.zip



Testing with another isotope: U-235
Isotope details: IsotopeInfo class for: U-235


INFO:pleiades.nuclear.manager:Resonance parameters written to endf_test/092-U-235.B-VIII.0.par


Output file: endf_test/092-U-235.B-VIII.0.par
CPU times: user 227 ms, sys: 189 ms, total: 416 ms
Wall time: 2.71 s


In [10]:
print("Cache content summary with filename patterns:")
cache_root = config.nuclear_data_cache_dir
if cache_root.exists():
    for source_dir in cache_root.iterdir():
        if source_dir.is_dir():
            print(f"\nSource: {source_dir.name}")
            for library_dir in source_dir.iterdir():
                if library_dir.is_dir():
                    files = list(library_dir.glob("*.dat"))
                    if files:
                        print(f"  Library: {library_dir.name}")
                        for file in files:
                            file_size = file.stat().st_size
                            print(f"    - {file.name} ({file_size / 1024:.2f} KB)")
                            # Highlight the pattern type
                            if "_" in file.name and file.name.split("_")[1].isdigit():
                                print(f"      Pattern: MAT_FIRST (non-zero-padded Z)")
                            else:
                                print(f"      Pattern: ELEMENT_FIRST (zero-padded Z)")

Cache content summary with filename patterns:

Source: DataSource.NNDC

Source: DataSource.IAEA
  Library: EndfLibrary.ENDF_B_VIII_1
    - n_092-U-238_9237.dat (14016.15 KB)
      Pattern: ELEMENT_FIRST (zero-padded Z)
  Library: EndfLibrary.ENDF_B_VIII_0
    - n_9228_92-U-235.dat (38863.12 KB)
      Pattern: MAT_FIRST (non-zero-padded Z)
    - n_9237_92-U-238.dat (15741.04 KB)
      Pattern: MAT_FIRST (non-zero-padded Z)
